___

In [ ]:
"""
Python 3, using Conda env: orthanc
@Author : Hasan Shaikh
@Email  : hasanshaikh3198@gmail.com
@GitHub : https://github.com/hash123shaikh
"""

In [1]:
# check_cmc_structures.py
import os
import glob
import pydicom

# Compatibility patch for newer pydicom versions
if not hasattr(pydicom, "read_file"):
    pydicom.read_file = pydicom.dcmread

from dcmrtstruct2nii import list_rt_structs

# === UPDATE THIS PATH ===
# Since code is in ~/Downloads/hasan and dataset is in subfolder:
base_dir = "./MAASTRO Dataset"

print("="*70)
print("🔍 CHECKING STRUCTURE NAMES IN CMC DATASET")
print("="*70 + "\n")
print(f"Dataset location: {os.path.abspath(base_dir)}\n")

# Check if path exists
if not os.path.exists(base_dir):
    print(f"❌ ERROR: Path does not exist!")
    print(f"Looking for: {os.path.abspath(base_dir)}")
    print("\nPlease check:")
    print("  1. Is the dataset folder name correct?")
    print("  2. Are you running this script from ~/Downloads/hasan ?")
    exit(1)

# Dictionary to count structure names
structure_count = {}
patients_with_structures = []
patients_with_errors = []

# Check ALL patients
patient_folders = sorted([d for d in os.listdir(base_dir) 
                         if os.path.isdir(os.path.join(base_dir, d))])

print(f"Found {len(patient_folders)} patient folders\n")
print("-"*100)

for i, patient_id in enumerate(patient_folders, 1):
    patient_path = os.path.join(base_dir, patient_id)
    
    # Look for RTSTRUCT folder (case-insensitive)
    subdirs = [d for d in os.listdir(patient_path) if os.path.isdir(os.path.join(patient_path, d))]
    subdirs_lower = {d.lower(): d for d in subdirs}
    
    if "rtstruct" not in subdirs_lower:
        print(f"[{i:3d}] {patient_id}: ⚠️  No RTSTRUCT folder (found: {subdirs})")
        continue
    
    rtstruct_folder = os.path.join(patient_path, subdirs_lower["rtstruct"])
    
    # Find DICOM files in RTSTRUCT
    dcm_files = glob.glob(os.path.join(rtstruct_folder, "*.dcm"))
    
    if not dcm_files:
        print(f"[{i:3d}] {patient_id}: ⚠️  No .dcm files in RTSTRUCT")
        continue
    
    try:
        # List all structures
        structures = list_rt_structs(dcm_files[0])
        
        # Print patient and their structures
        print(f"[{i:3d}] {patient_id}: {structures}")
        
        # Count structures
        for struct in structures:
            structure_count[struct] = structure_count.get(struct, 0) + 1
        
        patients_with_structures.append((patient_id, structures))
        
    except Exception as e:
        print(f"[{i:3d}] {patient_id}: ❌ Error - {e}")
        patients_with_errors.append((patient_id, str(e)))

print("-"*70)

# === SUMMARY ===
print("\n" + "="*70)
print("📊 SUMMARY")
print("="*70)
print(f"\nTotal patients checked: {len(patient_folders)}")
print(f"✅ Successfully read: {len(patients_with_structures)}")
print(f"❌ Errors: {len(patients_with_errors)}")

if structure_count:
    print(f"\n📋 STRUCTURE NAMES FOUND (and their frequency):")
    print("-"*70)
    for struct_name, count in sorted(structure_count.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / len(patients_with_structures)) * 100 if patients_with_structures else 0
        print(f"  '{struct_name}': {count} patients ({percentage:.1f}%)")

# === RECOMMENDATIONS ===
print("\n" + "="*70)
print("💡 RECOMMENDATIONS FOR YOUR SCRIPT")
print("="*70)

gtv_structures = [s for s in structure_count.keys() if "GTV" in s.upper()]

if gtv_structures:
    print("\nGTV structure names found:")
    for gtv in gtv_structures:
        count = structure_count[gtv]
        print(f"  - '{gtv}': {count} patients")
    
    if len(gtv_structures) == 1:
        print(f"\n✅ Use this in your main script (lines 68-73):")
        print(f"\n   selected_structure = None")
        print(f"   for s in structures:")
        print(f"       if s.upper() == '{gtv_structures[0].upper()}':")
        print(f"           selected_structure = s")
        print(f"           break")
    else:
        print(f"\n⚠️  Multiple GTV types found. Choose an option:")
        print(f"\n   Option 1: Use ANY GTV (takes first one found):")
        print(f"   ----------------------------------------")
        print(f"   selected_structure = None")
        print(f"   for s in structures:")
        print(f"       if 'GTV' in s.upper():")
        print(f"           selected_structure = s")
        print(f"           break")
        print(f"\n   Option 2: Use SPECIFIC GTV (e.g., GTV-1):")
        print(f"   ----------------------------------------")
        print(f"   selected_structure = None")
        print(f"   for s in structures:")
        print(f"       if s.upper() == 'GTV-1':  # Change to your preferred GTV")
        print(f"           selected_structure = s")
        print(f"           break")
else:
    print("\n⚠️  No GTV structures found!")
    print("Check if your dataset uses different naming.")
    if structure_count:
        print("\nAvailable structures:")
        for s in sorted(structure_count.keys()):
            print(f"  - '{s}'")

# Show any patients with errors
if patients_with_errors:
    print(f"\n⚠️  Patients with errors ({len(patients_with_errors)}):")
    for patient_id, error in patients_with_errors[:10]:  # Show first 10
        print(f"  - {patient_id}: {error}")
    if len(patients_with_errors) > 10:
        print(f"  ... and {len(patients_with_errors) - 10} more")

print("\n" + "="*70)
print("✅ DONE! Use the recommendations above to update your main script.")
print("="*70 + "\n")


Please cite:
Thomas Phil, Thomas Albrecht, Skylar Gay, & Mathis Ersted Rasmussen. (2023). Sikerdebaard/dcmrtstruct2nii: dcmrtstruct2nii v5 (Version v5). Zenodo. https://doi.org/10.5281/zenodo.4037864

🔍 CHECKING STRUCTURE NAMES IN CMC DATASET

Dataset location: /home/radiomicsserver/Downloads/hasan/MAASTRO Dataset

Found 137 patient folders

----------------------------------------------------------------------------------------------------
[  1] HN1004: ['Spinal-Cord', 'Neck-Right', 'Neck-Left', 'Submandibular-Gland-Right', 'Submandibular-Gland-Left', 'Parotid-Right', 'Parotid-Left', 'Oral-Cavity', 'Medulla-Oblongata', 'Brain', 'GTV-2', 'GTV-1']
[  2] HN1006: ['Spinal-Cord', 'Neck-Right', 'Neck-Left', 'Submandibular-Gland-Right', 'Submandibular-Gland-Left', 'Parotid-Right', 'Parotid-Left', 'Oral-Cavity', 'Medulla-Oblongata', 'Cochlea-Right', 'Cochlea-Left', 'Brain', 'GTV-3', 'GTV-2', 'GTV-1']
[  3] HN1022: ['Spinal-Cord', 'Neck-Right', 'Neck-Left', 'Submandibular-Gland-Right', 'Subma